# Libraries

In [1]:
import numpy as np

In [2]:
import tensorflow as tf
import keras

Using TensorFlow backend.


In [3]:
import os

In [4]:
from PIL import Image

# PATH TO DATA. CHANGE IT

In [5]:
data_path = r'D:\Scolaire\UdeM\IFT_6135\Assignment1\data'

# Data generator (for Keras)

In [60]:
def fetch_image(ID, which):
    ids = int(ID)
    dir_path = os.path.join(data_path, which + 'set')
    if which=='train':
        if ids > 9999:
            ids = ids - 10000
            clas = 'Dog'
        else:
            clas = 'Cat'
        with Image.open(os.path.join(dir_path, clas, str(ids)+'.'+clas+'.jpg')) as im:
            im_data = np.asarray(im)
    elif which=='test':
        with Image.open(os.path.join(dir_path, 'test', str(ids)+'.jpg')) as im:
            im_data = np.asarray(im)
    else:
        raise AssertionError("Wrong value in fetch image")
        
    s = im_data.shape
    if len(s) == 2:  # ie gray scale
        new_array = np.empty((*s, 3), dtype='int')
        for i in range(s[0]):
            for j in range(s[1]):
                new_array[i,j] = np.repeat(np.array([im_data[i,j]]), 3)
        return new_array
    else:
        return im_data

In [61]:
class DataGeneratorTrain(keras.utils.Sequence):
    
    def __init__(self, list_IDs, labels, batch_size=32, dim=(64,64), n_channels=3,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
          np.random.shuffle(self.indexes)
            
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = fetch_image(ID, 'train')

            # Store class
            y[i] = self.labels[ID]

        if self.n_classes > 2:  
            return X, keras.utils.to_categorical(y, num_classes=self.n_classes)
        else:
            return X, y

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

# Settings

In [62]:
# Parameters
params = {'dim': (64,64),
          'batch_size': 64,
          'n_classes': 2,
          'n_channels': 3,
          'shuffle': True}
val_size = 0.2

# Datasets
total_ids = np.hstack((np.arange(1, 10000), np.arange(10001, 20000)))
number_ids = len(total_ids)
labels = dict()
for ID in total_ids:
    labels[str(ID)] = int(ID>10000)
partition = dict()
np.random.seed(0)
np.random.shuffle(total_ids)
partition['train'] = [str(x) for x in total_ids[:int(number_ids*(1-val_size))]]
partition['validation'] = [str(x) for x in total_ids[int(number_ids*(1-val_size)):]]

# Generators
training_generator = DataGeneratorTrain(partition['train'], labels, **params)
validation_generator = DataGeneratorTrain(partition['validation'], labels, **params)

# Keras architecture

In [63]:
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [64]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [76]:
class CNN1:
    
    def __init__(self):
        np.random.seed(0)
        tf.set_random_seed(0)
        self.model = keras.models.Sequential()
        self.history = None
        
        # Layer 1
        self.model.add(Conv2D(32, kernel_size=3, strides=1, padding='same', use_bias=True,
                              activation='relu', input_shape=(64,64,3), data_format='channels_last'))
        self.model.add(MaxPooling2D(2, padding='valid', data_format='channels_last'))
        
        # Layer 2
        self.model.add(Conv2D(64, kernel_size=3, strides=1, padding='same', use_bias=True,
                              activation='relu', input_shape=(32,32,64), data_format='channels_last'))
        self.model.add(MaxPooling2D(2, padding='valid', data_format='channels_last'))
        
        # Layer 3
        self.model.add(Conv2D(128, kernel_size=3, strides=1, padding='same', use_bias=True,
                              activation='relu', input_shape=(16,16,128), data_format='channels_last'))
        self.model.add(MaxPooling2D(2, padding='valid', data_format='channels_last'))
        
        # Layer 4
        self.model.add(Flatten())
        self.model.add(Dense(512, use_bias=True, activation='relu'))
        
        # Layer 5
        self.model.add(Dense(1, use_bias=True, activation='sigmoid'))
        
        sgd = keras.optimizers.SGD(lr=0.01)
        self.model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
        
        self.model.summary()
    
    def train(self, ep, early=10):
        earlystop = EarlyStopping(patience=early)
        learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                                    patience=5, 
                                                    verbose=1, 
                                                    factor=0.5, 
                                                    min_lr=0.00001)
        callbacks = [earlystop, learning_rate_reduction]
        
        self.history = self.model.fit_generator(generator=training_generator,
                                                validation_data=validation_generator, 
                                                epochs=ep, callbacks=callbacks)
        # use_multiprocessing=True, workers=6,

In [77]:
mod1 = CNN1()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 8, 8, 128)         0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 8192)              0         
__________

In [78]:
mod1.train(15)

Epoch 1/15
249/249 [==============================] - 131s 525ms/step - loss: 7.9205 - acc: 0.5021 - val_loss: 8.0957 - val_acc: 0.4922
Epoch 2/15
249/249 [==============================] - 132s 528ms/step - loss: 7.9372 - acc: 0.5021 - val_loss: 8.1158 - val_acc: 0.4909
Epoch 3/15
249/249 [==============================] - 133s 533ms/step - loss: 7.9352 - acc: 0.5023 - val_loss: 8.1038 - val_acc: 0.4917
Epoch 4/15
249/249 [==============================] - 133s 533ms/step - loss: 7.9452 - acc: 0.5016 - val_loss: 8.0998 - val_acc: 0.4919
Epoch 5/15
249/249 [==============================] - 131s 526ms/step - loss: 7.9342 - acc: 0.5023 - val_loss: 8.1118 - val_acc: 0.4912
Epoch 6/15
249/249 [==============================] - 138s 554ms/step - loss: 7.9422 - acc: 0.5018 - val_loss: 8.1118 - val_acc: 0.4912

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.05000000074505806.
Epoch 7/15
249/249 [==============================] - 131s 527ms/step - loss: 7.9352 - acc: 0.5023 - val_